# **Classification Model BUilding**

---

## **Import necessary modules**

In [1]:
# basic library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from dotenv.main import load_dotenv

# sql connection library
import mysql.connector as mysql

# feature engineering library
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# model building library
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier,
                            AdaBoostClassifier,
                            GradientBoostingClassifier)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# model evaluation library
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# model tuning library
from sklearn.model_selection import GridSearchCV

# save model
import joblib

# load env
load_dotenv()

# ignore warning
warnings.filterwarnings('ignore')


## **Load the data from database**

In [2]:
def load_data():
    # connect to database
    conn = mysql.connect(user = os.getenv('MYSQL_USER'), password = os.getenv('MYSQL_PASSWORD'), host = os.getenv('MYSQL_HOST'), database = os.getenv('MYSQL_DATABASE'))
    # convert to dataframe
    df = pd.read_sql('SELECT * FROM admission', con = conn)
    # close connection
    conn.close()
    return df

df = load_data()

### Show the top 5 rows

In [3]:
df.head()

,application_id,gender,international,gpa,major,race,gmat,work_exp,work_industry,admission
0,1,Female,False,3.30,Business,Asian,620.0,3.0,Financial Services,Admit
1,2,Male,False,3.28,Humanities,Black,680.0,5.0,Investment Management,
2,3,Female,True,3.30,Business,,710.0,5.0,Technology,Admit
3,4,Male,False,3.47,STEM,Black,690.0,6.0,Technology,
4,5,Male,False,3.35,STEM,Hispanic,590.0,5.0,Consulting,


---

## **Data Preprocessing**

### Fill the blank values with appropriate labels

In [22]:
df['race'] = df['race'].replace('', 'Unknown')
df['admission'] = df['admission'].replace('', 'Reject')

In [23]:
df.head()

,application_id,gender,international,gpa,major,race,gmat,work_exp,work_industry,admission
0,1,Female,False,3.30,Business,Asian,620.0,3.0,Financial Services,Admit
1,2,Male,False,3.28,Humanities,Black,680.0,5.0,Investment Management,Reject
2,3,Female,True,3.30,Business,Unknown,710.0,5.0,Technology,Admit
3,4,Male,False,3.47,STEM,Black,690.0,6.0,Technology,Reject
4,5,Male,False,3.35,STEM,Hispanic,590.0,5.0,Consulting,Reject


### Prepare the features and target column

In [5]:
X = df.drop(columns=['admission'], axis = 1)
y = df['admission']

In [6]:
X.head()

,application_id,gender,international,gpa,major,race,gmat,work_exp,work_industry
0,1,Female,False,3.30,Business,Asian,620.0,3.0,Financial Services
1,2,Male,False,3.28,Humanities,Black,680.0,5.0,Investment Management
2,3,Female,True,3.30,Business,Unknown,710.0,5.0,Technology
3,4,Male,False,3.47,STEM,Black,690.0,6.0,Technology
4,5,Male,False,3.35,STEM,Hispanic,590.0,5.0,Consulting


In [7]:
y.head()

0     Admit
1    Reject
2     Admit
3    Reject
4    Reject
Name: admission, dtype: object

### Unique categories in categorical features

In [8]:
categorical_features = X.select_dtypes(include=['object']).columns

for feature in categorical_features:
    print(f'{feature} : {df[feature].unique()}', end='\n\n')

gender : ['Female' 'Male']

international : ['False' 'True']

major : ['Business' 'Humanities' 'STEM']

race : ['Asian' 'Black' 'Unknown' 'Hispanic' 'White' 'Other']

work_industry : ['Financial Services' 'Investment Management' 'Technology' 'Consulting'
 'Nonprofit/Gov' 'PE/VC' 'Health Care' 'Investment Banking' 'Other'
 'Retail' 'Energy' 'CPG' 'Real Estate' 'Media/Entertainment']



> **There are fewer unique categories in each column except for work industry, so we'll try both one-hot and ordinal encoding**

### Handle numeric and categorical features

In [9]:
# create column transformer to preprocess data

# numerical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

# categorical features
categorical_features = X.select_dtypes(include=['object']).columns

df['race'] = df['race'].replace('', 'Unknown')
df['admission'] = df['admission'].replace('', 'Reject')

# numerical pipeline
num_pipeline = Pipeline(
    steps =[

    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())

    ]
)

# categorical pipeline
cat_pipeline = Pipeline(
    steps= [

        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)),
        ('scaler', StandardScaler())

    ]
)

# full pipeline using column transformer
preprocessor = ColumnTransformer(
    [

    ('num', num_pipeline, numeric_features),
    ('cat', cat_pipeline, categorical_features)
    
    ]
)

In [10]:
# fit and transform preprocessor on input data
X = preprocessor.fit_transform(X)

# fit and transform label encoder on target data
le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
y[:25]

array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0])

In [12]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4335, 26), (1859, 26), (4335,), (1859,))

In [14]:
# perform oversampling for training data
sm = SMOTE()

X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)

In [21]:
y_train_oversampled[:25]

array([0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1])

---

## **Model Building**

In [15]:
# create a function to evaluate model
def evaluate_model(actual, pred):
    '''
    Function to evaluate model performance

    Args:
    actual : Actual/True label
    pred : Predicted label

    Returns:
    accuracy : accuracy score
    matrix : confusion matrix
    report : classification report
    '''
    accuracy = accuracy_score(actual, pred)
    matrix = confusion_matrix(actual, pred)
    report = classification_report(actual, pred)
    
    return accuracy, matrix, report

In [16]:
models = {
    'Logistic Regression' : LogisticRegression(),
    'Random Forest' : RandomForestClassifier(),
    'AdaBoost' : AdaBoostClassifier(),
    'Gradient Boosting' : GradientBoostingClassifier(),
    'SVM' : SVC(),
    'Decision Tree' : DecisionTreeClassifier(),
    'KNN' : KNeighborsClassifier(),
    'XGBoost' : XGBClassifier()
}

model_list = []
train_accuracy_list = []
test_accuracy_list = []

for i in range(len(list(models))):
    model = list(models.values()) [i]
    model.fit(X_train_oversampled, y_train_oversampled)

    # make predictions
    y_train_pred = model.predict(X_train_oversampled)
    y_test_pred = model.predict(X_test)

    # evaluate train and test performance
    train_model_accuracy, train_model_matrix, train_model_report = evaluate_model(y_train_oversampled, y_train_pred)
    test_model_accuracy, test_model_matrix, test_model_report = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print(f"- Accuracy: \n{train_model_accuracy}", end='\n\n')
    print(f"- Confusion matrix: \n{train_model_matrix}", end='\n\n')
    print(f"- Classification report: \n{train_model_report}", end='\n\n')
    train_accuracy_list.append(train_model_accuracy)

    print('----------------------------------')
    
    print('Model performance for Test set')
    print(f"- Accuracy: \n{test_model_accuracy}", end='\n\n')
    print(f"- Confusion matrix: \n {test_model_matrix}", end='\n\n')
    print(f"- Classification report: \n {test_model_report}", end='\n\n')
    test_accuracy_list.append(test_model_accuracy)
    
    print('='*35)
    print('\n')

Logistic Regression
Model performance for Training set
- Accuracy: 
0.6192423003543199

- Confusion matrix: 
[[2010  555 1104]
 [ 609 2518  542]
 [ 875  506 2288]]

- Classification report: 
              precision    recall  f1-score   support

           0       0.58      0.55      0.56      3669
           1       0.70      0.69      0.69      3669
           2       0.58      0.62      0.60      3669

    accuracy                           0.62     11007
   macro avg       0.62      0.62      0.62     11007
weighted avg       0.62      0.62      0.62     11007


----------------------------------
Model performance for Test set
- Accuracy: 
0.6503496503496503

- Confusion matrix: 
 [[ 158   55   86]
 [ 241 1038  246]
 [   9   13   13]]

- Classification report: 
               precision    recall  f1-score   support

           0       0.39      0.53      0.45       299
           1       0.94      0.68      0.79      1525
           2       0.04      0.37      0.07        35

    a

---

## **Models Results**

In [17]:
results = pd.DataFrame(list(zip(model_list, test_accuracy_list, train_accuracy_list)), columns=['Model', 'Test Accuracy', 'Train Accuracy']).sort_values(by=['Test Accuracy', 'Train Accuracy'], ascending=False)

results

,Model,Test Accuracy,Train Accuracy
7,XGBoost,0.829478,0.997093
1,Random Forest,0.819258,1.000000
3,Gradient Boosting,0.806885,0.902880
5,Decision Tree,0.797741,1.000000
2,AdaBoost,0.771920,0.790406
4,SVM,0.714363,0.904606
6,KNN,0.669177,0.924230
0,Logistic Regression,0.650350,0.619242


---

> **Most of the models are overfitting and LR is underfitting, so we'll choose AdaBoost for this problem statement**

In [18]:
# prediction for actual data
ada_boost = AdaBoostClassifier()
ada_boost.fit(X_train, y_train)

y_pred = ada_boost.predict(X_test)

score = accuracy_score(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy of AdaBoost is {score}', end='\n\n')
print(f'Confusion Matrix: \n {matrix}')

Accuracy of AdaBoost is 0.8187197417966648

Confusion Matrix: 
 [[  66  232    1]
 [  65 1456    4]
 [   6   29    0]]


In [19]:
# prediction for oversampled data
ada_boost = AdaBoostClassifier()
ada_boost.fit(X_train_oversampled, y_train_oversampled)

y_pred = ada_boost.predict(X_test)

score = accuracy_score(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)

print(f'Accuracy of AdaBoost is {score}', end='\n\n')
print(f'Confusion Matrix: \n {matrix}')

Accuracy of AdaBoost is 0.7719203873050027

Confusion Matrix: 
 [[ 211   77   11]
 [ 271 1218   36]
 [  12   17    6]]
